# Agentics Mini Tutorial

Agentics provides the implementation of **AG**, a powerful datatype that connects
LLMs to Pydantic objects and enables **logical transduction**.

---

## Installation

```bash
!uv pip install agentics-py

In [ ]:
! uv pip install agentics-py

! uv pip install agentics-py


import os
from pathlib import Path
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    from google.colab import drive
  
    drive.mount("/content/drive")
    from google.colab import userdata
    
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")
else:

    CURRENT_PATH = os.getcwd()
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

base = Path(CURRENT_PATH)

Using Python 3.12.9 environment at: /Users/gliozzo/Code/agentics911/agentics/.venv
Audited 1 package in 16ms
In Colab: False


## Use Agentics as Lists

Agentics objects (`AG`) can be used similarly to Python lists, allowing you to store and manage collections of states. You can append new elements using the `.append()` method, and access all states via the `.states` attribute.

For example, after creating an empty `AG` object, you can add elements:

In [ ]:
from agentics import AG
my_first_agentics = AG()

print("The agentics is empty :", len(my_first_agentics))

 ## Add elements to the list
my_first_agentics.append("Alfio")
## internally, agentics stores the elements in the attribute states
my_first_agentics.states += ["Naweed" , "Junkyuu"] 

print("The agentics now has more instances :",len(my_first_agentics))

try:
    print("this triggers an error")
    my_first_agentics = my_first_agentics + my_first_agentics
except:
    my_first_agentics.states= my_first_agentics.states + my_first_agentics.states
    print("This is the right way to concetenate two agentics. Be careful, the states should be instances of the same atype")
    my_first_agentics.pretty_print()

print("Iterating over agentics:") 
for state in  my_first_agentics:
    print(state)

print("Be careful, the AG itself is not a list :" , my_first_agentics) 



2025-09-29 07:00:04.627 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


The agentics is empty : 0
The agentics now has more instances : 3
this triggers an error
This is the right way to concetenate two agentics. Be careful, the states should be instances of the same atype
Atype : None
Alfio
...

Naweed
...

Junkyuu
...

Alfio
...

Naweed
...

Junkyuu
...


Atype : None
Alfio
...

Naweed
...

Junkyuu
...

Alfio
...

Naweed
...

Junkyuu
...


Iterating over agentics:
Alfio
Naweed
Junkyuu
Alfio
Naweed
Junkyuu
Be careful, the AG itself is not a list : atype=None crew_prompt_params={'role': 'Task Executor', 'goal': 'You execute tasks', 'backstory': 'You are always faithful and provide only fact based answers.', 'expected_output': 'Described by Pydantic Type'} instructions='Generate an object of the specified type from the following input.' llm=<crewai.llm.LLM object at 0x31e0d1b80> max_iter=3 prompt_template=None reasoning=None skip_intentional_definition=False states=['Alfio', 'Naweed', 'Junkyuu', 'Alfio', 'Naweed', 'Junkyuu'] tools=None transduce_fields=None 

## Atypes

Agentics supports **typed AGs** using Pydantic models, enabling you to enforce schema validation and structure on the states stored in an AG. This is useful when you want all elements in your AG to follow a specific format or contain certain fields.

To define a typed AG:

1. **Create a Pydantic model** that describes the schema for your states.
2. **Instantiate an AG** with the `atype` parameter set to your Pydantic model.
3. **Add instances** of your model to the AG. Only objects matching the schema will be accepted.

This approach ensures data consistency and allows you to leverage Pydantic's validation features within Agentics workflows.

For example, you can define a `Movie` type and create an AG that only accepts `Movie` instances as its states. See the next cell for a practical demonstration.

In [ ]:
from pydantic import BaseModel
from typing import Optional

# Define the Movie Pydantic model for use with Agentics AG
class Movie(BaseModel):
    movie_name: Optional[str] = None
    genre: Optional[str] = None
    description: Optional[str] = None


movies = AG(atype=Movie)
movies.append(Movie(movie_name="La dolce vita"))
movies.pretty_print()

2025-09-29 07:00:04.638 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Atype : <class '__main__.Movie'>
movie_name: La dolce vita
genre: null
description: null


Atype : <class '__main__.Movie'>
movie_name: La dolce vita
genre: null
description: null




## Extending and Merging AGs
AGs can evolve by adding new fields or combining with other AGs to form richer schemas.

### Add attributes
Use `.add_attribute()` to dynamically extend the schema of an AG.  
This operation mutates the AG in place.

In [ ]:
movies = AG(atype=Movie)
movies.append(Movie(movie_name="La dolce vita"))
movies.pretty_print()

print("adding a new attribute to the type and rebinding the object")
movies.add_attribute("email", 
                     description="Write an email to tell a fried about this movie")

movies.pretty_print()
print("Note that the AG changed")


2025-09-29 07:00:04.646 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Atype : <class '__main__.Movie'>
movie_name: La dolce vita
genre: null
description: null


adding a new attribute to the type and rebinding the object
Atype : <class '__main__.Movie'>
movie_name: La dolce vita
genre: null
description: null


Atype : <class '__main__.Movie'>
movie_name: La dolce vita
genre: null
description: null


Note that the AG changed


### Subtypes
You can project an AG onto a subset of its fields, e.g. `movies("title", "genre")`.  
This creates a new AG without modifying the original.

In [20]:
movies_subtype = movies("movie_name", "genre")
print("This is a subtype")
movies_subtype.pretty_print()

print("This is the original type.\nNote that the AG didn't change after subtype")
movies.pretty_print()
print("Note that the AG didn't change after subtyping it")

This is a subtype
Atype : <class 'agentics.core.agentics.movie_name_genre'>
movie_name: The Shawshank Redemption
genre: Drama, Crime

movie_name: The Godfather
genre: Drama, Crime

movie_name: The Godfather Part II
genre: Drama, Crime

movie_name: Schindler's List
genre: Drama, History, War

movie_name: 12 Angry Men
genre: Drama

movie_name: Spirited Away
genre: Animation, Family, Fantasy

movie_name: The Dark Knight
genre: Drama, Action, Crime, Thriller

movie_name: Dilwale Dulhania Le Jayenge
genre: Comedy, Drama, Romance

movie_name: The Green Mile
genre: Fantasy, Drama, Crime

movie_name: Parasite
genre: ''

movie_name: Pulp Fiction
genre: ''

movie_name: Your Name.
genre: ''

movie_name: 'The Lord of the Rings: The Return of the King'
genre: ''

movie_name: Forrest Gump
genre: ''

movie_name: The Good, the Bad and the Ugly
genre: ''

movie_name: Seven Samurai
genre: ''

movie_name: GoodFellas
genre: ''

movie_name: Interstellar
genre: ''

movie_name: Grave of the Fireflies
genre: 

### Merge AGs
You can merge AGs of different types (e.g., `Movie` with `Director`), combining their states into a new AG with a union of fields.  
On field conflicts, values from the right-hand AG’s states take precedence.

In [21]:
# Define a new Pydantic type for directors
class Director(BaseModel):
    director_name: Optional[str] = None

# Merge movies (Movie type) with directors (Director type)
# The result AG will have fields from both Movie and Director
prod_movies = movies.merge(
    AG(atype=Director, states=[Director(director_name="Fellini")])
)

# Add another movie to the original AG
movies.append(Movie(movie_name="Superman"))

print("Merging AGs will combine states:")

# Merge with one director state; the single director is aligned with each movie
movies.merge(
    AG(atype=Director, states=[Director(director_name="Fellini")])
).pretty_print()

# Merge with two director states; directors are aligned by index with movies
# If the AGs are different lengths, extra states are still included
movies.merge(
    AG(atype=Director, states=[
        Director(director_name="Fellini"),
        Director(director_name="Donner")
    ])
).pretty_print()

2025-09-29 09:19:41.948 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'
2025-09-29 09:19:41.950 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'
2025-09-29 09:19:41.951 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'
2025-09-29 09:19:41.952 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'
2025-09-29 09:19:41.958 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'
2025-09-29 09:19:41.966 | DEBUG    | agentics.core.llm_connection

Merging AGs will combine states:
Atype : <class 'agentics.core.agentics.Movie__merge__Director'>
movie_name: The Shawshank Redemption
genre: Drama, Crime
description: Imprisoned in the 1940s for the double murder of his wife and her lover,
  upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where
  he puts his accounting skills to work for an amoral warden. During his long stretch
  in prison, Dufresne comes to be admired by the other inmates -- including an older
  prisoner named Red -- for his integrity and unquenchable sense of hope.
director_name: Fellini

movie_name: The Godfather
genre: Drama, Crime
description: Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American
  Corleone crime family. When organized crime family patriarch, Vito Corleone barely
  survives an attempt on his life, his youngest son, Michael steps in to take care
  of the would-be killers, launching a campaign of bloody revenge.
director_name: null

movie_name: Th

'Atype : <class \'agentics.core.agentics.Movie__merge__Director\'>\nmovie_name: The Shawshank Redemption\ngenre: Drama, Crime\ndescription: Imprisoned in the 1940s for the double murder of his wife and her lover,\n  upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where\n  he puts his accounting skills to work for an amoral warden. During his long stretch\n  in prison, Dufresne comes to be admired by the other inmates -- including an older\n  prisoner named Red -- for his integrity and unquenchable sense of hope.\ndirector_name: Fellini\n\nmovie_name: The Godfather\ngenre: Drama, Crime\ndescription: Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American\n  Corleone crime family. When organized crime family patriarch, Vito Corleone barely\n  survives an attempt on his life, his youngest son, Michael steps in to take care\n  of the would-be killers, launching a campaign of bloody revenge.\ndirector_name: Donner\n\nmovie_name: The Godfathe

## Import Agentics for Json and CSV 

Agentics AG objects can be easily imported from and exported to CSV and JSONL formats. This enables seamless integration with tabular and structured data workflows.

- **CSV Import/Export:**  
    Use `AG.from_csv("path/to/file.csv")` to create an AG from a CSV file. The schema (`atype`) can be inferred automatically or provided explicitly.
- **JSONL Import/Export:**  
    Use `AG.to_jsonl("path/to/file.jsonl")` to export, and `AG.from_jsonl("path/to/file.jsonl")` to import AG objects in JSON Lines format.

This functionality allows you to move data between Agentics and other tools with minimal effort.

In [22]:
# Create a new AG object from the provided csv file
movies = AG.from_csv(base / "data/movies.csv", max_rows=3)
movies.pretty_print()

# Note that the atype has been automatically inffered
print("Imported Type", movies.atype)

# Reloading same file by providing atype
movies = AG.from_csv(base /"data/movies.csv", atype=Movie)

# Note that just a subset of the attributes have been imported
print("Provided Type" , movies.atype)

# agentics can be exported and imported from jsonl objects
movies.to_jsonl(base /"data/movies.jsonl")
movies= AG.from_jsonl(base / "data/movies.jsonl")

# note this type is different from what imported from csv
print("Imported atype from jsonl: ", movies.atype)

2025-09-29 09:19:50.667 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'
2025-09-29 09:19:50.669 | DEBUG    | agentics.core.agentics:from_csv:303 - Importing Agentics of type Movie from CSV data/movies.csv
2025-09-29 09:19:50.671 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'
2025-09-29 09:19:50.671 | DEBUG    | agentics.core.agentics:to_jsonl:409 - Exporting 100 states or atype <class '__main__.Movie'> to data/movies.jsonl
2025-09-29 09:19:50.678 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Atype : <class 'agentics.core.atype.AType#movie_name:genre:descriptionOptional'>
movie_name: The Shawshank Redemption
genre: Drama, Crime
description: Imprisoned in the 1940s for the double murder of his wife and her lover,
  upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where
  he puts his accounting skills to work for an amoral warden. During his long stretch
  in prison, Dufresne comes to be admired by the other inmates -- including an older
  prisoner named Red -- for his integrity and unquenchable sense of hope.

movie_name: The Godfather
genre: Drama, Crime
description: Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American
  Corleone crime family. When organized crime family patriarch, Vito Corleone barely
  survives an attempt on his life, his youngest son, Michael steps in to take care
  of the would-be killers, launching a campaign of bloody revenge.

movie_name: The Godfather Part II
genre: Drama, Crime
description: In the

## Logical Transduction

Once an AG is initialized with an atype, Agentics can **transduce** any string of text and/or pydantic object into that type.  If a list of strings is provided, they are processed asynchronously.

## Untyped transduction

If no target atype is provided, transduction works as a regular llm call, where the input text or pydantic object is given to the LLM and the output is the LLM response. In this use case, agentics provides an off the shelp **async scale-out framework for LLM calls**. 

Note that no AType is specified, the output of transduction is alist of strings. So it is not recommended to use this notation for transduction algebra. In addition, Unconstrained trnasduction tends to me less efficient as it requires the LLM to guess the type of output required, often resulting in verbose and unecessary information . 

In [23]:
import time

questions = [
    "What are the benefits of using Agentic AI for data workflows?",
    "Will AI improve working conditions for the middle class?",
    "How can Agentic AI enhance decision-making in finance?",
    # "What risks should companies consider when adopting AI agents?",
    # "Can AG objects integrate with existing data pipelines?",
    # "Who won the latest FIFA worldcup",
]
start = time.time()
answers = await (AG() << questions)
end = time.time()

for question, answer in zip(questions, answers):
    print(f"Question: {question}\nAnswer{answer}\n")
print(f"Uncostrained transduction done in {end-start} seconds")


2025-09-29 09:21:09.617 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'
2025-09-29 09:21:09.617 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Output()

Question: What are the benefits of using Agentic AI for data workflows?
Answer## Why Agentic AI Is a Game‑Changer for Data Workflows

| Benefit | What It Means for Your Data Pipeline | Real‑World Impact |
|---------|--------------------------------------|-------------------|
| **End‑to‑end automation** | Agents can **orchestrate** every step—from data ingestion, cleaning, and transformation to model training, validation, and deployment—without manual hand‑offs. | A retail analytics team cuts the nightly ETL window from 6 hours to <30 minutes. |
| **Dynamic decision‑making** | Agents continuously **monitor** data quality, resource usage, and model performance, then **re‑route** or **re‑configure** tasks on the fly (e.g., switch to a backup data source when the primary API throttles). | A fraud‑detection pipeline automatically falls back to a lighter‑weight model during traffic spikes, keeping latency <200 ms. |
| **Self‑learning & adaptation** | Using reinforcement‑learning or meta‑lear

### Transduction into Atype

You can define a target schema with Pydantic (e.g., `Answer`) and transduce text into it.  
The LLM output is parsed and validated into the fields `answer`, `justification`, and `confidence`.  
Note that the output is more clean and organized, and the time required to execute the transduction is one order of magnitude lower. 

In [24]:
# Define a Pydantic model for a structured answer
class Answer(BaseModel):
    # The main response text
    answer: Optional[str] = None
    # An explanation or reasoning behind the answer
    justification: Optional[str] = None
    # A numeric confidence score (e.g. from 0.0 to 1.0)
    confidence: Optional[float] = None

# Transduce a natural language question into the structured Answer schema
start= time.time()
answers = await (AG(atype=Answer) << questions)
end= time.time()
print(f"Typed transduction done in {end-start} seconds")
answers.pretty_print()

2025-09-29 09:22:04.021 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Output()

Typed transduction done in 2.947697162628174 seconds
Atype : <class '__main__.Answer'>
answer: 'Agentic AI brings several key benefits to data workflows: it automates repetitive
  tasks, adapts dynamically to changing data conditions, scales efficiently with growing
  data volumes, reduces manual effort and human error, accelerates insight generation,
  enables continuous learning and improvement, and facilitates seamless integration
  across heterogeneous data sources and tools.'
justification: "Automation frees analysts from routine processing, while the agentic\
  \ nature allows the AI to make context\u2011aware decisions (e.g., selecting appropriate\
  \ models or cleaning strategies) as data evolves. Scalability is achieved through\
  \ autonomous resource management, and the reduction of manual steps lowers the risk\
  \ of mistakes. Faster, self\u2011optimizing pipelines deliver insights more quickly,\
  \ and the built\u2011in learning loops let the system improve over time, e

'Atype : <class \'__main__.Answer\'>\nanswer: \'Agentic AI brings several key benefits to data workflows: it automates repetitive\n  tasks, adapts dynamically to changing data conditions, scales efficiently with growing\n  data volumes, reduces manual effort and human error, accelerates insight generation,\n  enables continuous learning and improvement, and facilitates seamless integration\n  across heterogeneous data sources and tools.\'\njustification: "Automation frees analysts from routine processing, while the agentic\\\n  \\ nature allows the AI to make context\\u2011aware decisions (e.g., selecting appropriate\\\n  \\ models or cleaning strategies) as data evolves. Scalability is achieved through\\\n  \\ autonomous resource management, and the reduction of manual steps lowers the risk\\\n  \\ of mistakes. Faster, self\\u2011optimizing pipelines deliver insights more quickly,\\\n  \\ and the built\\u2011in learning loops let the system improve over time, ensuring\\\n  \\ the work

### Transduction Between AGs

You can control transduction more precisely by converting **from one AG to another**:
- The **source AG** provides the input states (rendered via the prompt).
- The **target AG** defines the output schema and validation.
- Agentics renders each source state → sends it to the LLM → parses into the target type.

This pattern is ideal when you want consistent, structured outputs from heterogeneous inputs while keeping prompts and schema separate.

### Transduction Between AGs  
Here we convert product reviews (`ProductReview`) into sentiment summaries (`SentimentSummary`).  
The source AG provides the reviews, and the target AG enforces structured outputs (positive/neutral/negative with a reason).  

In [25]:
from typing import Optional, Literal
from pydantic import BaseModel
from agentics import AG

# Source schema: product reviews
class ProductReview(BaseModel):
    reviewer: Optional[str] = None
    text: Optional[str] = None
    stars: Optional[int] = None

# Target schema: summarized sentiment
class SentimentSummary(BaseModel):
    customer_sentiment: Optional[Literal["positive", "neutral", "negative"]] = None
    reason: Optional[str] = None

# Example reviews
reviews = [
    ProductReview(reviewer="Alice", text="Excellent quality and fast delivery!", stars=5),
    ProductReview(reviewer="Bob", text="Okay, but packaging was damaged", stars=3),
    ProductReview(reviewer="Carol", text="Terrible, broke after one use", stars=1),
]

# Create source and target AGs
source = AG(atype=ProductReview, states=reviews)
target = AG(atype=SentimentSummary)

# Transduce reviews into sentiment summaries
sentiments = await (target << source)
sentiments.pretty_print()

2025-09-29 09:23:38.900 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'
2025-09-29 09:23:38.901 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Output()

Atype : <class '__main__.SentimentSummary'>
customer_sentiment: positive
reason: The review praises excellent quality and fast delivery.

customer_sentiment: negative
reason: packaging was damaged

customer_sentiment: negative
reason: Product broke after one use




"Atype : <class '__main__.SentimentSummary'>\ncustomer_sentiment: positive\nreason: The review praises excellent quality and fast delivery.\n\ncustomer_sentiment: negative\nreason: packaging was damaged\n\ncustomer_sentiment: negative\nreason: Product broke after one use\n\n"

### Self-Transduction  

You can transduce within the same AG type by selecting different subsets of fields.  
This is useful for projecting, comparing, or enriching dataframes and state graphs without changing the original AG.  

In [27]:
movies = AG.from_csv(base / "data/movies.csv",atype=Movie)
movies.filter_states(start =10, end =20)

self_transductions = await movies.self_transduction(["movie_name","description"],["genre"])
print(self_transductions.pretty_print())

2025-09-29 09:25:26.573 | DEBUG    | agentics.core.agentics:from_csv:303 - Importing Agentics of type Movie from CSV data/movies.csv
2025-09-29 09:25:26.574 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Output()

Atype : <class '__main__.Movie'>
movie_name: Pulp Fiction
genre: Science Fiction
description: A burger-loving hit man, his philosophical partner, a drug-addled gangster's
  moll and a washed-up boxer converge in this sprawling, comedic crime caper. Their
  adventures unfurl in three stories that ingeniously trip back and forth in time.

movie_name: Your Name.
genre: Science Fiction
description: "High schoolers Mitsuha and Taki are complete strangers living separate\
  \ lives. But one night, they suddenly switch places. Mitsuha wakes up in Taki\u2019\
  s body, and he in hers. This bizarre occurrence continues to happen randomly, and\
  \ the two must adjust their lives around each other."

movie_name: 'The Lord of the Rings: The Return of the King'
genre: War drama
description: "As armies mass for a final battle that will decide the fate of the world--and\
  \ powerful, ancient forces of Light and Dark compete to determine the outcome--one\
  \ member of the Fellowship of the Ring is 

### Customizing Transduction  

You can fine-tune how logical transduction works by configuring:  

- **LLMs** – choose the underlying language model to run the transduction.  
- **Instructions** – add task-specific guidance for the LLM.  
- **Prompt Templates** – control how inputs are rendered into prompts.  
- **Few-Shot Examples** – provide examples to steer the model’s behavior.  
- **Verbose Options** – enable detailed logging and debug outputs.  

#### Task instructions
The example below illustrate how to provide a llm and task specific instructions to transduction

In [12]:
questions_answering_ag=AG(atype=Answer,
                          llm=AG.get_llm_provider("watsonx"),
                          instructions= "Answer in italian")

print((await (questions_answering_ag << questions)).pretty_print())


Output()

Atype : <class '__main__.Answer'>
answer: "Le aziende devono considerare diversi rischi quando adottano agenti IA, tra\
  \ cui:\n1. **Bias e discriminazione**: gli algoritmi possono riflettere pregiudizi\
  \ presenti nei dati di addestramento, portando a decisioni ingiuste.\n2. **Privacy\
  \ e protezione dei dati**: gli agenti IA spesso elaborano grandi quantit\xE0 di\
  \ informazioni sensibili, aumentando il rischio di violazioni o uso improprio.\n\
  3. **Sicurezza e vulnerabilit\xE0**: gli agenti possono essere bersaglio di attacchi\
  \ informatici o manipolazioni (ad es. avvelenamento dei dati), compromettendo l\u2019\
  integrit\xE0 dei sistemi.\n4. **Responsabilit\xE0 legale**: \xE8 difficile attribuire\
  \ la responsabilit\xE0 per errori o danni causati da decisioni automatizzate.\n\
  5. **Trasparenza e spiegabilit\xE0**: la \u201Cblack box\u201D degli algoritmi pu\xF2\
  \ ostacolare la comprensione delle decisioni, creando sfiducia tra stakeholder e\
  \ autorit\xE0 di 

#### Prompt templates

Prompt templates enable greater customization of your transductions by providing a langchain style abstraction to render pydantic objects into input prompts for the agent. 



In [13]:
questions_answering_ag=AG(atype=Answer)

dow_jones_data=AG.from_csv("data/dow_jones.csv")
dow_jones_data =dow_jones_data.get_random_sample(0.002)
dow_jones_data.prompt_template="what happened to the financial markets in {date}?"
answers = await (questions_answering_ag << dow_jones_data)
print(answers.pretty_print())



2025-09-29 07:00:58.241 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'
2025-09-29 07:00:58.256 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Output()

 Received None or empty response from LLM call.

 An unknown error occurred. Please check the details below.

 Error details: Invalid response from LLM call - None or empty.

 An unknown error occurred. Please check the details below.

 Error details: Invalid response from LLM call - None or empty.

2025-09-29 07:01:14.168 | DEBUG    | agentics.core.async_executor:execute:65 - retrying 1 state(s), attempt 1
2025-09-29 07:01:21.302 | DEBUG    | agentics.core.async_executor:execute:65 - retrying 1 state(s), attempt 2
2025-09-29 07:01:27.529 | DEBUG    | agentics.core.agentics:__lshift__:575 - ⚠️  1 states have not been transduced


Atype : <class '__main__.Answer'>
answer: "On June 11, 2010, major U.S. equity markets posted modest gains. The Dow\
  \ Jones Industrial Average rose about 0.5% to roughly 11,500 points, the S&P 500\
  \ increased about 0.6% to around 1,155, and the Nasdaq Composite climbed about 0.7%\
  \ to near 2,500. The rally was driven by easing fears over the European sovereign\u2011\
  debt crisis after Greece\u2019s first bailout and by expectations that the Federal\
  \ Reserve would keep interest rates low."
justification: "Historical market data for June\u202F11\u202F2010 show the Dow up\
  \ ~57 points (\u22480.5%), the S&P\u202F500 up ~7 points (\u22480.6%), and the Nasdaq\
  \ up ~18 points (\u22480.7%). Contemporary news reports (e.g., Reuters, Bloomberg)\
  \ highlighted that the gains were linked to reduced anxiety about the European debt\
  \ situation following Greece\u2019s bailout and to the Fed\u2019s accommodative\
  \ stance, which supported risk assets."
confidence: 0.78

ans

### Few-Shot Learning

Agentics natively supports **few-shot examples**: you can preload the **target AG** with
gold states (examples of the desired output). During transduction, these examples steer
the LLM toward consistent labels/structures.

Below, we load movies from CSV, manually seed the first 10 labels as few-shots, and then
transduce the rest.

In [ ]:
# Zero Shots= all output states are blank
movies = AG.from_csv(base / "data/movies.csv",atype=Movie)
movies.filter_states(start =10, end =20)
self_transductions = await movies.self_transduction(["movie_name","description"],["genre"])
print("this is zero shot")
self_transductions.pretty_print()

# Few shots = the first 10 states are used as examples
movies = AG.from_csv(base / "data/movies.csv",atype=Movie)
movies.filter_states(start =0, end =20)
self_transductions = await movies.self_transduction(["movie_name","description"],["genre"])

# printing only transduced states
movies.filter_states(start =10, end =20)
print("this is the output with 10 shots")
self_transductions.pretty_print()

2025-09-29 09:29:48.180 | DEBUG    | agentics.core.agentics:from_csv:303 - Importing Agentics of type Movie from CSV data/movies.csv
2025-09-29 09:29:48.181 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Output()

2025-09-29 09:30:39.717 | DEBUG    | agentics.core.agentics:from_csv:303 - Importing Agentics of type Movie from CSV data/movies.csv
2025-09-29 09:30:39.718 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Output()

this is zero shot
Atype : <class '__main__.Movie'>
movie_name: Pulp Fiction
genre: Crime
description: A burger-loving hit man, his philosophical partner, a drug-addled gangster's
  moll and a washed-up boxer converge in this sprawling, comedic crime caper. Their
  adventures unfurl in three stories that ingeniously trip back and forth in time.

movie_name: Your Name.
genre: Western
description: "High schoolers Mitsuha and Taki are complete strangers living separate\
  \ lives. But one night, they suddenly switch places. Mitsuha wakes up in Taki\u2019\
  s body, and he in hers. This bizarre occurrence continues to happen randomly, and\
  \ the two must adjust their lives around each other."

movie_name: 'The Lord of the Rings: The Return of the King'
genre: Fantasy
description: "As armies mass for a final battle that will decide the fate of the world--and\
  \ powerful, ancient forces of Light and Dark compete to determine the outcome--one\
  \ member of the Fellowship of the Ring is re

this is the output with 10 shots
Atype : <class '__main__.Movie'>
movie_name: The Shawshank Redemption
genre: Drama, Action, Crime, Thriller
description: Imprisoned in the 1940s for the double murder of his wife and her lover,
  upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where
  he puts his accounting skills to work for an amoral warden. During his long stretch
  in prison, Dufresne comes to be admired by the other inmates -- including an older
  prisoner named Red -- for his integrity and unquenchable sense of hope.

movie_name: The Godfather
genre: Drama, Adventure, Fantasy, Action
description: Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American
  Corleone crime family. When organized crime family patriarch, Vito Corleone barely
  survives an attempt on his life, his youngest son, Michael steps in to take care
  of the would-be killers, launching a campaign of bloody revenge.

movie_name: The Godfather Part II
genre: Western,

'Atype : <class \'__main__.Movie\'>\nmovie_name: The Shawshank Redemption\ngenre: Drama, Action, Crime, Thriller\ndescription: Imprisoned in the 1940s for the double murder of his wife and her lover,\n  upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where\n  he puts his accounting skills to work for an amoral warden. During his long stretch\n  in prison, Dufresne comes to be admired by the other inmates -- including an older\n  prisoner named Red -- for his integrity and unquenchable sense of hope.\n\nmovie_name: The Godfather\ngenre: Drama, Adventure, Fantasy, Action\ndescription: Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American\n  Corleone crime family. When organized crime family patriarch, Vito Corleone barely\n  survives an attempt on his life, his youngest son, Michael steps in to take care\n  of the would-be killers, launching a campaign of bloody revenge.\n\nmovie_name: The Godfather Part II\ngenre: Western, Drama, Actio

## Tool Usage  

Agentics integrates seamlessly with the **MCP ecosystem**, allowing AGs to call external tools during transduction.  In addition to that, they also allows the use of CrewAI tools, as the underlying transduction framework is currently based on crewAI agents. 

This makes it easy to fetch, process, or enrich data dynamically while keeping results structured. 

In the following example we illustrate the use of Duck Duck Go search to improve the information gathering of historical market data. 

In [15]:
from crewai.tools import tool
from ddgs import DDGS


## Define a Crew AI tool to get news for a given date using the DDGS search engine
@tool("web_search")
def web_search(query: str) -> str:
    """Fetch web search results for the given query using DDGS."""
    return str(DDGS().text(query, max_results=10))


questions_answering_ag.verbose_agent = True
questions_answering_ag.tools=[web_search]
dow_jones_data.filter_states(end=1)
answers = await (questions_answering_ag << dow_jones_data)
print(answers.pretty_print())

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 1e3a6fc8-6383-43cc-834d-cd00aafdb164                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Task Executor                                                                                           │
│                                                                                                                 │
│  Task:                                                                                                          │
│  Your task is to transduce a source Pydantic Object into the specified Output type. Generate only slots that    │
│  are logically deduced from the input information, otherwise live then null.                                    │
│                                                                                                                 │
│  Read carefully the following instructions for executing your task:                                             │
│  Generate an object of the specified type from the following input. SOURCE:                                     │
│  what happened to the financial markets in 2010-06-11?                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Task Executor                                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "answer": "On June 11, 2010 the U.S. equity markets posted modest gains. The Dow Jones Industrial Average    │
│  closed at 11,332.70, up about 0.1% (≈+12 points). The S&P 500 finished at 1,155.5, up roughly 0.2% (≈+2.3      │
│  points), and the Nasdaq Composite rose about 0.3% to 2,511.5. The day was relatively calm after the May 6      │
│  flash‑crash, with investors focusing on earnings reports and the ongoing European sovereign‑debt concerns.     │
│  Commodity prices slipped (crude oil fell around 1%), while Treasury yields edged higher, reflecting a slight   │
│  risk‑off tilt.",                                                                                               │
│    "justification": "These figures are reported by multiple market‑data sources for the date June 11, 2010,     │
│  including Yahoo Finance historical quotes (Dow 11,332.70; S&P 1,155.5; Nasdaq 2,511.5) and MarketWatch daily   │
│  summary, which note the modest gains and the broader market context following the May flash‑crash and          │
│  European debt worries.",                                                                                       │
│    "confidence": 0.86                                                                                           │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: fdb81d87-ca17-4efb-a883-78070b7f0667                                                                     │
│  Agent: Task Executor                                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 1e3a6fc8-6383-43cc-834d-cd00aafdb164                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│    "answer": "On June 11, 2010 the U.S. equity markets posted modest gains. The Dow Jones Industrial Average    │
│  closed at 11,332.70, up about 0.1% (≈+12 points). The S&P 500 finished at 1,155.5, up roughly 0.2% (≈+2.3      │
│  points), and the Nasdaq Composite rose about 0.3% to 2,511.5. The day was relatively calm after the May 6      │
│  flash‑crash, with investors focusing on earnings reports and the ongoing European sovereign‑debt concerns.     │
│  Commodity prices slipped (crude oil fell around 1%), while Treasury yields edged higher, reflecting a slight   │
│  risk‑off tilt.",                                                                                               │
│    "justification": "These figures are reported by multiple market‑data sources for the date June 11, 2010,     │
│  including Yahoo Finance historical quotes (Dow 11,332.70; S&P 1,155.5; Nasdaq 2,511.5) and MarketWatch daily   │
│  summary, which note the modest gains and the broader market context following the May flash‑crash and          │
│  European debt worries.",                                                                                       │
│    "confidence": 0.86                                                                                           │
│  }                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Atype : <class '__main__.Answer'>
answer: "On June 11, 2010 the U.S. equity markets posted modest gains. The Dow Jones\
  \ Industrial Average closed at 11,332.70, up about 0.1% (\u2248+12 points). The\
  \ S&P 500 finished at 1,155.5, up roughly 0.2% (\u2248+2.3 points), and the Nasdaq\
  \ Composite rose about 0.3% to 2,511.5. The day was relatively calm after the May\
  \ 6 flash\u2011crash, with investors focusing on earnings reports and the ongoing\
  \ European sovereign\u2011debt concerns. Commodity prices slipped (crude oil fell\
  \ around 1%), while Treasury yields edged higher, reflecting a slight risk\u2011\
  off tilt."
justification: "These figures are reported by multiple market\u2011data sources for\
  \ the date June 11, 2010, including Yahoo Finance historical quotes (Dow 11,332.70;\
  \ S&P 1,155.5; Nasdaq 2,511.5) and MarketWatch daily summary, which note the modest\
  \ gains and the broader market context following the May flash\u2011crash and European\
  \ debt w